# 检索器可以用于检索文档，协助大模型回答问题

In [8]:
from langchain_openai import ChatOpenAI
# 通过langchain设置chatapi 
llm = ChatOpenAI()

# 不设置环境变量的方式 ,非常不建议这么做
#from langchain_openai import ChatOpenAI
 
#llm = ChatOpenAI(api_key="...")

In [2]:
# 加载要索引的数据。为此，我们将使用WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 将其索引到向量存储中。
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [6]:
# 构建我们的索引
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

现在，我们在向量存储中索引了这些数据，我们将创建一个检索链。该链将接收一个输入问题，查找相关文档，然后将这些文档连同原始问题一起传递给LLM，并要求它回答原始问题。

首先，让我们设置链来接收一个问题和检索到的文档，并生成一个回答。

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:
 
<context>
{context}
</context>
 
Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

如果我们想的话，可以直接传递文档来运行它：

In [10]:
from langchain_core.documents import Document
 
document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help with testing by allowing you to visualize test results.'

然而，我们希望文档首先来自我们刚刚设置的检索器。
这样，我们就可以使用检索器动态选择最相关的文档并将其传递给给定的问题。

In [11]:
from langchain.chains import create_retrieval_chain
 
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

现在，我们可以调用此链。这将返回一个词典-LLM的响应在answer键中。

In [12]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])
 
# LangSmith提供了几个功能，可以帮助进行测试:...

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk, create custom evaluations, compare results for different configurations side-by-side, provide a playground environment for rapid iteration and experimentation, collect data on application performance in real-world scenarios during beta testing, gather human feedback on application responses, attach feedback scores to logged traces, send runs to annotation queues for inspection, add runs as examples to datasets to expand test coverage, closely inspect key data points, monitor key metrics over time, perform online evaluations and automations on production traces in near real-time, group traces from a single conversation for easier tracking, and conduct A/B testing on changes in prompt, model, or retrieval strategy.
